In [1]:
!pip install syft==0.2.0a2

     |████████████████████████████████| 337kB 2.9MB/s 
     |████████████████████████████████| 1.8MB 9.4MB/s 
     |████████████████████████████████| 10.1MB 20.0MB/s 
     |████████████████████████████████| 471kB 61.9MB/s 
     |████████████████████████████████| 256kB 59.2MB/s 
     |████████████████████████████████| 773.1MB 23kB/s 
     |████████████████████████████████| 204kB 63.1MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for zstd: filename=zstd-1.4.4.0-cp36-cp36m-linux_x86_64.whl size=1131222 sha256=02678832132dadb7c3677eca680fe1167e032710adb02aebe4859408b4cb9d35
  Stored in directory: /root/.cache/pip/wheels/ce/0d/5a/3c2ccf5522d1cd3ecdeb2af8c9889179d7a204376b1d5c042d
Successfully built zstd
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
  Found existing installation: torchvi

In [2]:
!git clone https://github.com/Intelligent-Systems-Lab/Private-AI.git
!unzip /content/Private-AI/Federated_Learning2/twitter-user-gender-classification.zip -d /content

Cloning into 'Private-AI'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 63 (delta 22), reused 55 (delta 16), pack-reused 0
Unpacking objects: 100% (63/63), done.
Archive:  /content/Private-AI/Federated_Learning2/twitter-user-gender-classification.zip
  inflating: /content/gender-classifier-DFE-791531.csv  


In [0]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
import time
import datetime

from sklearn.metrics import roc_auc_score

import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd import Variable

import os
# for dirname, _, filenames in os.walk('/content'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [0]:
import sys
sys.path.append("/content/Private-AI/Federated_Learning")

In [0]:
STOPWORDS = set([])  # set(stopwords.words('english'))
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in STOPWORDS])
    return text

def tokenize(text, word_to_idx):
    tokens = []
    for word in text.split():
        tokens.append(word_to_idx[word])
    return tokens


def pad_and_truncate(messages, max_length=30):
    features = np.zeros((len(messages), max_length), dtype=int)
    for i, sms in enumerate(messages):
        if len(sms):
            features[i, -len(sms):] = sms[:max_length]
    return features

In [0]:
data = pd.read_csv('/content/gender-classifier-DFE-791531.csv', encoding='latin-1')
data = pd.concat([data.gender,data.description,data.text],axis=1)
data.dropna(inplace=True,axis=0)
data.gender = [1 if each == "female" else 0 for each in data.gender] 

In [7]:
data

,gender,description,text
0,0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,0,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,0,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,1,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...
...,...,...,...
20045,1,(rp),"@lookupondeath ...Fine, and I'll drink tea too..."
20046,0,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...
20047,0,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...
20048,1,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...


In [0]:
data_gender = []

for t in data.gender:
    v = np.zeros(2)
    if t==0:
        v[0] = 1
    else:
        v[1] = 1
    data_gender.append(v)

In [9]:
data_gender[:10]

[array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([0., 1.]),
 array([0., 1.]),
 array([1., 0.]),
 array([1., 0.]),
 array([0., 1.]),
 array([0., 1.])]

In [0]:
data.description = data.description.apply(clean_text)

In [11]:
words = set((' '.join(data.description)).split())
print(len(words))

33672


In [0]:
word_to_idx = {word: i for i, word in enumerate(words, 1)}
tokens = data.description.apply(lambda x: tokenize(x, word_to_idx))
inputs = pad_and_truncate(tokens)

#labels = np.array((data.gender).astype(int))

labels = np.array(data_gender)
#labels = data.gender

In [13]:
labels.shape

(16224, 2)

In [14]:
print(inputs[:2])
print(labels[:2])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0 15980 33188  2967 14982  8519]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0  1375  9036 14273 27735 22338
  11875 10387 32213   786 25928 27253]]
[[1. 0.]
 [1. 0.]]


In [0]:
VOCAB_SIZE = int(inputs.max()) + 1
# Training params
EPOCHS = 15
CLIP = 5 # gradient clipping - to avoid gradient explosion (frequent in RNNs)
lr = 0.1
BATCH_SIZE = 128

# Model params
EMBEDDING_DIM = 100
HIDDEN_DIM = 12
DROPOUT = 0.2

In [0]:
labels = torch.tensor(labels)
inputs = torch.tensor(inputs)

pct_test = 0.2

train_labels = labels[:-int(len(labels)*pct_test)]
train_inputs = inputs[:-int(len(labels)*pct_test)]

test_labels = labels[-int(len(labels)*pct_test):]
test_inputs = inputs[-int(len(labels)*pct_test):]

In [17]:
from handcrafted_GRU import GRU
model = GRU(vocab_size=VOCAB_SIZE, output_size=2, hidden_dim=HIDDEN_DIM, embedding_dim=EMBEDDING_DIM, dropout=DROPOUT)
model.to('cuda:0')

GRU(
  (dropout): Dropout(p=0.2, inplace=False)
  (embedding): Embedding(33673, 100)
  (gru_cell): GRUCell(
    (fc_ir): Linear(in_features=100, out_features=12, bias=True)
    (fc_hr): Linear(in_features=12, out_features=12, bias=True)
    (fc_iz): Linear(in_features=100, out_features=12, bias=True)
    (fc_hz): Linear(in_features=12, out_features=12, bias=True)
    (fc_in): Linear(in_features=100, out_features=12, bias=True)
    (fc_hn): Linear(in_features=12, out_features=12, bias=True)
  )
  (fc): Linear(in_features=12, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [0]:
class dataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs =inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.inputs[idx],self.labels[idx]

In [0]:
train_dataloader = DataLoader(dataset(train_inputs,train_labels),
                               batch_size=BATCH_SIZE, 
                               shuffle=True,
                               num_workers=4, 
                               pin_memory=True)

test_dataloader = DataLoader(dataset(test_inputs,test_labels),
                               batch_size=BATCH_SIZE, 
                               shuffle=True,
                               num_workers=4, 
                               pin_memory=True)

In [0]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [0]:
def train(epoch,dloader):
    for step, (x,y) in enumerate(dloader):
        data = Variable(x).cuda()
        target = Variable(y).cuda()
        h = torch.Tensor(np.zeros((BATCH_SIZE, HIDDEN_DIM))).to('cuda:0')
        output,_ = model(data,h)
        loss = criterion(output, target.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step==0:
            start = time.time()
            ti = 0
        elif step==50:
            ti = time.time()-start #total time = ti*(length/100)
            #print(ti)
            ti = ti*(len(dloader)/50)
        if step % 50 == 0:
            second = ti*(((len(dloader)-step)/len(dloader)))#*(5-epoch)*(4-fnum)
            print('Ep: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}\t Remain : {} '.
                     format(epoch+1, 
                            step * len(data), 
                            len(dloader.dataset),
                            50.*step/len(dloader), 
                            loss.data.item(),
                            datetime.timedelta(seconds = int(second))))
        torch.cuda.empty_cache()
    #print("Finish")

In [0]:
def val(dloader):
    los = []
    acc_num = 0
    for step, (x, y) in enumerate(dloader):
        data = Variable(x).cuda()
        target = Variable(y).cuda()
        h = torch.Tensor(np.zeros((BATCH_SIZE, HIDDEN_DIM))).to('cuda:0')
        with torch.no_grad():
            output,_ = model(data,h)
        
        loss = criterion(output, target.float())
        los.append(loss.item())
        
        out = np.argmax(target.cpu().data.numpy().squeeze(),axis = 1)
        pre = np.argmax(output.cpu().data.numpy().squeeze(),axis = 1)
        #print(pre)
        #print(out)
        acc_num += (out == pre).sum()
        
        # if step %50 == 0:
        #     print('[{}/{} ({:.1f}%)]'.format(step * len(data), 
        #                                 len(dloader.dataset),
        #                                 50.*step/len(dloader)))

        torch.cuda.empty_cache()
    print("Acc : [{}/{} ({:.1f}%)]".format(acc_num,
                                         len(dloader.dataset),
                                         100.*acc_num/len(dloader.dataset)))
    los = np.array(los)
    avg_val_loss = los.sum()/len(los)
    print("Avg val loss: {:.8f}".format(loss))

In [23]:
for epoch in range(EPOCHS):
    if epoch %1==0:
        lr = lr*0.9
        optimizer = optim.SGD(model.parameters(), lr=lr)
    model.train()
    train(epoch, train_dataloader)
    model.eval()
    val(test_dataloader)
    print("===============================")

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Ep: 1 [0/12980 (0%)]	 Loss: 0.726588	 Remain : 0:00:00 
Ep: 1 [6400/12980 (25%)]	 Loss: 0.719115	 Remain : 0:00:02 
Ep: 1 [12800/12980 (49%)]	 Loss: 0.710934	 Remain : 0:00:00 
Acc : [1862/3244 (57.4%)]
Avg val loss: 0.71094203
Ep: 2 [0/12980 (0%)]	 Loss: 0.712067	 Remain : 0:00:00 
Ep: 2 [6400/12980 (25%)]	 Loss: 0.706095	 Remain : 0:00:01 
Ep: 2 [12800/12980 (49%)]	 Loss: 0.702237	 Remain : 0:00:00 
Acc : [1941/3244 (59.8%)]
Avg val loss: 0.70072275
Ep: 3 [0/12980 (0%)]	 Loss: 0.698231	 Remain : 0:00:00 
Ep: 3 [6400/12980 (25%)]	 Loss: 0.700376	 Remain : 0:00:01 
Ep: 3 [12800/12980 (49%)]	 Loss: 0.698860	 Remain : 0:00:00 
Acc : [1950/3244 (60.1%)]
Avg val loss: 0.69620836
Ep: 4 [0/12980 (0%)]	 Loss: 0.696358	 Remain : 0:00:00 
Ep: 4 [6400/12980 (25%)]	 Loss: 0.692558	 Remain : 0:00:02 
Ep: 4 [12800/12980 (49%)]	 Loss: 0.690425	 Remain : 0:00:00 
Acc : [1962/3244 (60.5%)]
Avg val loss: 0.70981836
Ep: 5 [0/12980 (0%)]	 Loss: 0.687259	 Remain : 0:00:00 
Ep: 5 [6400/12980 (25%)]	 Loss: 